<a href="https://colab.research.google.com/github/glamacles/notebooks/blob/main/gnns.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Graph Neural Networks and Emulators

## Unstructured Data

- CNNs are can be very effective for dealing with problems that have structured data on a  grid
- This makes CNNs ideal for raster datasets
- But some data is not well suited to a grid based representation
- Examples: Finite element meshes, social networks, particle systems, etc.

<div>
<img src="https://github.com/glamacles/notebooks/blob/main/images/0_KIKnUvzdIkp5zcDJ.jpg?raw=true" width="300"/>
</div>

<div>
<img src="https://github.com/glamacles/notebooks/blob/main/images/Left-to-right-uniform-mesh-as-commonly-used-in-finite-differences-scheme-triangle-mesh_W640.jpg?raw=true" width="300"/>
</div>



## Graph Neural Networks (GNNs)
- What is a graph?
- A set of vertices $V$ and edges $E$ defining connections among vertices
- Notably, grids correspond to a particular kind of graph structure with uniformly spaced vertices and consistent connectivity rules between nearby vertices
- Edges can be directed (an edge between vertex $i$ and vertex $j$ does not imply an edge between vertex $j$ and vertex $i$) or undirected

<div>
<img src="https://github.com/glamacles/notebooks/blob/main/images/local_emulators/graph.png?raw=1" width="400"/>
</div>

- GNNs operate on graph data structures, meaning inputs and outputs are defined on graphs
- This means a GNN takes in both the graph structure ($V$, $E$) as well as data defined on the graph
- Similarly, outputs include a graph data structure + data defined on that graph
- In many applications the input / output graph is the same, but features defined on vertices / edges are modified, but this isn't always true
- In general:
$$ (G_0, X_{V_0}, X_{E_0}) \to \text{GNN} \to (G_1, X_{V_1}, X_{E_1}) $$
-Here, $G_0 = (V_0, E_0)$ is the input graph and $G_1 = (V_1, E_1)$ is the output graph
- The input graph has vertex and edge-wise feature matrices $X_{V_0}$ and $X_{E_0}$ and analagously for the output graph
- This just means that each vertex / edge can have some associated feature vector
- Sometimes there are also graph level features as well. These are composite features associated with the whole graph or particular subgraphs

## **Components of a GNN**
- GNNs often have analgous components to CNNs
- These include **propogation modules** used to propogate information between nodes (these are usually defined similarly to convolutions in CNNs)
- **Sampling modules** are used on to reduce data on large graphs (an example is selecting a subset of each nodes neighborhood)
- Then, also similar to CNNs, there are **pooling modules**
- See [Zhou et al. 2020](https://www.sciencedirect.com/science/article/pii/S2666651021000012) for more information

### Propogation Modules

- We will focus on propogation operators in GNNs
- These operators, often modeled after convolutions, allow for transfer of infomration between graph nodes

### **Message Passing**
- Message passing allows information to propogate between adjacent nose
- GNNs often consist of a series of message passing steps
- For example, a message from vertex $j$ to vertex $i$ is given by
$$ \pmb{m}_{j,i} = \phi^{(k)} \left ( \pmb{x}_i^{(k-1)}, \pmb{x}_j^{(k-1)}, \pmb{e}_{j,i}^{(k-1)} \right )$$
- Here $\pmb{x}_i^{(k-1)}$ and $\pmb{x}_j^{(k-1)}$ are node features input into the message passing layer
- $\pmb{e}_{j,i}^{(k-1)}$ is the edge feature vector associated with the edge connecting node $j$ to $i$
- $\phi^{(k)}$ is a differentiable function, often an multi-layer perceptron (MLP)
-Once messages are formed, they are used to update node features:
$$ \mathbf{x}_i^{(k)} = \gamma^{(k)} \left( \mathbf{x}_i^{(k-1)}, \bigoplus_{j \in \mathcal{N}(i)} \, \pmb{m}_{j,i} \right) $$
- Here, $\mathbf{x}_i^{(k)}$ is the new feature vector for node $i$, $\gamma^{(k)}$ is a differentiable function (again usually an MLP), and $\bigoplus$ denotes an aggregation function
- The notation $\mathcal{N}(i)$ denotes the neighborhood of vertex $i$, or the set of all vertices connected to vertex $i$
- Common aggregation functions are mean, sum, and max

### Communication

- Each message passing layer means that information can transer 1 "hop"
- Each layer allows information to transfer over a larger neighborhood
- This is a slightly different approach to CNNs where convolutions are sometimes defined on larger neighborhoods


###  **Example: Graph Attention Networks**
- GATs fit into the message passing framework and use an attention mechanism as in transformers
- In an attention layer, first attention coefficients are computed
$$ e_{i,j} = a \left( W \pmb{x}_i^{(k-1)}, W \pmb{x}_j^{(k-1)} \right ) $$
- Then the attention coefficients are normalized in a neighborhood of a node $i$ via the softmax function
$$ \alpha_{i,j} = \text{softmax}_j(e_{i,j})  = \frac{\exp ( e_{i,j} )}{\sum_{k \in \mathcal{N}(i)} \exp ( e_{i,k} ) }$$
- Then the normalized attention coefficients are used to update node features
$$ \pmb{x}_i^{(k)} = \sigma \left ( \sum_{j \in \mathcal{N}(i)} \alpha_{i,j} W \pmb{x}_j^{(k-1)} \right ) $$
- Here $\sigma$ is your activation function
- This represents a single head of attention, but multi-head attention is often used
$$ \pmb{x}_i^{(k)} =  {\big\|}_{l=1}^L  \sigma^l \left ( \sum_{j \in \mathcal{N}(i)} \alpha_{i,j}^l W^l \pmb{x}_j^{(k-1)} \right ) $$
- $\big\|$ is the concatenation operator
- For more details see [Veličković et al.](https://arxiv.org/abs/1710.10903)

### **Use Case: Emulation**
- There are use cases of GNNs in a variety of fields, but we'll focus on their use for physical models
- Numerical models can be very slow to run and require a lot of computational resources
- For some tasks, like uncertainty quantification, we might want to perform many model runs
- This motivates the development of emulators, which are computationally inexpensive substitutes for numerical models
- GNNs have been demonstrated for emulating both particle based and mesh based numerical models
- For example, [Sanchez-Gonzalez et al. ](https://arxiv.org/pdf/2002.09405) apply GNNs to simulate particle systems based on the material point method

<div>
<img src="https://github.com/glamacles/notebooks/blob/main/images/Screenshot%20from%202025-06-04%2010-17-10.png?raw=true" width="800"/>
</div>

- Here, graph vertices represent particles with features like position, velocity, and material properties
- Edges are added dynamically based on distance between points so that nearby particles can interact
- Edges have features like the offset or distance between two particles
- Uses a message passing network as described above
- GNNs have also been applied to mesh based simulations as in [Pfaff et al.](https://arxiv.org/abs/2010.03409), which uses a similar approach
- This work highlights that there is a direct correspondence between a finite element mesh and a graph

# **Distinguishing Types of Emulators**
- There are many different approaches to emulation with different levels of generality
- Some emulators are domain specific, targeting a specific spatial / temporal domain, or a limited set of inputs
- Other emulators strive to be more general, requiring either no or limited retraining to apply to new spatial domains and time periods
- Both more specific and general purpose emulators have been developed to simulate glaciers
- For example, [He et al. (2023)](https://www.sciencedirect.com/science/article/abs/pii/S0021999123005235) construct a highly efficient emulator of Humbholdt Glacier in Greenland using a Deep-Operator Network (as covered by Mauro!)
- [Koo and Rahnemoonfar (2024)](https://www.cambridge.org/core/services/aop-cambridge-core/content/view/3BF933067DFFB16949CDADBFE95086AB/S0022143024000935a.pdf/div-class-title-graph-convolutional-network-as-a-fast-statistical-emulator-for-numerical-ice-sheet-modeling-div.pdf) uses a GNN to emulate ISSM for specific glaciers
- The Instructed Glacier Model is relatively general purpose emulator for simulating mountain glaciers

<br>

| Emulator              | Generality | Architecture |
| :---------------- | :------------------: | :------------------: |
| [He et al. (2023)](https://www.sciencedirect.com/science/article/abs/pii/S0021999123005235)      |   Glacier Specific  | Deep O-net |
| [Koo and Rahnemoonfar (2024)](https://www.cambridge.org/core/services/aop-cambridge-core/content/view/3BF933067DFFB16949CDADBFE95086AB/S0022143024000935a.pdf/div-class-title-graph-convolutional-network-as-a-fast-statistical-emulator-for-numerical-ice-sheet-modeling-div.pdf)       |   Glacier Specific   | GNN
| [IGM](https://www.cambridge.org/core/journals/journal-of-glaciology/article/deep-learning-speeds-up-ice-flow-modelling-by-several-orders-of-magnitude/748E962A103D2AF45F4CA8823C88C0C0)  | Not Glacier Specific | CNN  |

<br>
- Emulators like IGM often take advantage of network architectures that exploit local spatial relationships between variables like CNNs and GNNs
- Hence, we'll refer to them as local emulators

# **The Instructed Glacier Model**

- The Instructed Glacier Model ([IGM](https://www.cambridge.org/core/journals/journal-of-glaciology/article/deep-learning-speeds-up-ice-flow-modelling-by-several-orders-of-magnitude/748E962A103D2AF45F4CA8823C88C0C0)) drastically speeds up simulations of mountain glaciers
- Estimates ice flow velocity from geometry, basal traction coefficient, and the temperature dependent rate factor
- Approximates ice flow velocity using a fully convolutional network
- All inputs / outputs are defined on grid
- IGM resolves vertical velocity profile with 10 vertical spatial layers



<table>
  <tr>
    <td><img src="https://github.com/glamacles/notebooks/blob/main/images/local_emulators/igm2.png?raw=1" width="800"></td>
    <td>

| Variable              | Symbol |
| :---------------- | :------: |
| Ice Thickness       |   $h$   |
| Surface Elevation         |   $s$   |
| Rate Factor   |  $A$   |
| Basal Traction Coefficient |  $ c$  |
| Grid Resolution |  $H$  |
| Velocity X-component | $u$ |
| Velocity Y-component | $v$ |

  
  </tr>
</table>

- Note that IGM only emulates the velocity solve in the ice dynamics model
- Solutions to the mass continuity equation and heat equation are computed efficiently on the GPU using finite differences

## **Physics Informed Loss Function**

- Solving for ice velocity can be posed as minimization problem
- The varitional principle for momentum balance is given by
  $$ J(\pmb{u}) = \int_{\Omega} \left [ \frac{2n}{n+1} \eta (\dot{\epsilon}^2) \dot{\epsilon}^2 + \rho g \pmb{u} \cdot \nabla{s}   \right ] \; d \Omega + \int_{\Gamma_b} \frac{c}{m+1} (\pmb{u} \cdot \pmb{u})^{\frac{m+1}{2}} \; d \Gamma $$
- Taking the first variation $\delta J$ yields the weak form of the first order ice flow equations
- This serves as a natural way of defining a loss function for training the CNN
- In IGM the strain tensor $\dot{\epsilon}$ is simplified using the first order Blatter-Pattyn approxmiation

## **Training**

- IGM has been geared toward simulating mountain glaciers and ice fields
- Hence training data involves simulations on mountainous terrain in the European Alps generated by CsFlow or PISM
- IGM does not need training data per se, but it does need a wide variety of input scenarios to train a generallly applicable emulator
- IGM has a collection of pretrained emulators
- It also allows for dynamic retraining to improve accuracy if needed

# **IGM-Like GNN Emulator**

- IGM is designed for an ice flow model that uses a uniform grid
- Many models, like ISSM, are defined on an unstructured mesh
- This motivates using an alternative approach that can handle models defined on non-uniform meshes
- Intuitively, finite element meshes can be easily interpreted as graphs
- Mesh vertices correspond to graph vertices
- Mesh edges correspond to graph edges
- In [Koo and Rahnemoonfar (2024)](https://www.cambridge.org/core/services/aop-cambridge-core/content/view/3BF933067DFFB16949CDADBFE95086AB/S0022143024000935a.pdf/div-class-title-graph-convolutional-network-as-a-fast-statistical-emulator-for-numerical-ice-sheet-modeling-div.pdf), the finite element mesh used by ISSM is translated directly to a graph
- Variables, defined on mesh nodes become vertex features


<img src="https://github.com/glamacles/notebooks/blob/main/images/Screenshot%20from%202025-06-04%2012-10-08.png?raw=true" width="800">

## **A GNN Emulator for SpecEIS**
- [SpecEIS](https://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=10221861) is a another ice flow model that uses particular finite elements for stability and mass conservation
- To emulate SpecEIS, it turns out to be more convenient to define a graph based on the dual mesh
- Graph vertices are defined at cell centers
- Graph edges are defined between cell when two edges share a common edge on the standard mesh

<img src="https://github.com/glamacles/notebooks/blob/main/images/Screenshot%20from%202025-06-04%2013-16-20.png?raw=true" width="400">

- Vertex features (defined on cell centers) include the basal traction coefficient $\beta$ and ice thickness $H$
- Edge features include jumps in the spatial coordinate $\pmb{x}$, bed $B$, and thickness $H$.
- These are analagous to directional derivatives
- The output of the model is velocity, which is defined on edges for compatibility with SpecEIS

### **Training**

- The GNN SpecEIS emulator is trained on simulations of mountain glaciers throughout Montana
- Similar to IGM
- Perhaps due to using unstructured grids, the emulator requires additional measures for stability in time-dependent simulations
- This includes adding noise to the training data and performing "rollouts"
- This is where multiple emulator time steps are used to evaluate the loss function
- With these tricks, the results are mostly pretty good

In [ ]:
from IPython.display import HTML

HTML('<iframe width="1000" height="600" src="https://www.youtube.com/embed/-q8n2ivSxAs" frameborder="0" allow="accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>')

### **Some Big Challenges**
- The GNN approach on unstructured grids seem to have some innate challenges not encountered by IGM
- For instance, tests on ideal geometry do not exhibit the expected symmetry
- More effort is required during training to ensure stability in time-dependent simulations
- "Wiggles"
